In [2]:
import cv2
import numpy as np

#El flujo optico esta relacionado con la funcion Lukas-Kanade. Movimiento de puentos especifios o regiones. Tiene que ver una verificacion entre un frame y otro.
#El flujo optico es el seguimiento que se esta realizando
    #1ro detectar esquinas. (Shitomasi)
    #2do se hace el flujo. (Lukas Kanade)
#Deteccion de esquinas - Shitomasi.

#Control y deteccion de esquina
#Parametros Shitomasi
#corner_track_params=dict(maxCorners=10 'Cantidad de esquinas a detectar',qualityLevel=0.3 'nivel de calidad',minDistance=7 'distancia entre esquinas',blockSize=7 'tamano de kernel')
corner_track_params=dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)


#Parametros Lukas Kanade
#lk_params=dict(winSize=(200,200)'tamano de ventana',maxLevel=2 'nivel maximo de deteccion. 2 nivel las esquinas detectadas', criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03) 'Criterio de verificacion de flujo optimo. EPS valor de error. 10 iteracion y epsilo 0.3')
lk_params=dict(winSize=(200,200),maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_COUNT,10,0.03))

cap=cv2.VideoCapture(0)
#Lectura del frame anterior
ret,prev_frame=cap.read() 
prev_frame_gray=cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)#Convertir el frameanterior a escala de grises.
prevPts=cv2.goodFeaturesToTrack(prev_frame_gray, mask=None, **corner_track_params) #Captura previa de las esquinas con shitomasi.
mask=np.zeros_like(prev_frame) #Matriz verificadora de puntos #Enmascarado. zeros_likes, matriz verificadora de puntos.

while True:
    #Lectura del frame actual
    ret, frame=cap.read() 
    frame_gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    nextPts, status,error=cv2.calcOpticalFlowPyrLK(prev_frame_gray, frame_gray,prevPts,None,**lk_params) #Captura del flujo optico con Lukas Canada. Siempre trabajar con la funcion piramidal.
    good_new=nextPts[status==1] #Frame actual bueno. Buenas esquinas de seguimiento. Obtener los mejores arrays a nivel de estatus.El mejor array tiene el valor de 1.
    good_prev=prevPts[status==1] #Frame anterior bueno.
    
    #Generacion de linea para ver el moviento
    for i,(new,prev) in enumerate(zip(good_new, good_prev)): #La compresion para eliminar lo malo y quedarse con lo bueno de las esquinas.
        x_new,y_new=new.ravel()#Ravel aplana la informacion. Matriz a Vector.
        x_prev,y_prev=prev.ravel()
        #Dibujar la linea
        mask=cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3) 
        #Dibujar circulo
        frame=cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)    
    img=cv2.add(frame,mask) #Agragar las capas del frame y mask
    cv2.imshow("Frame",img)
    k=cv2.waitKey(30) & 0xFF
    if k==27:
        break
    prev_frame_gray=frame_gray.copy()
    prevPts=good_new.reshape(-1,1,2) #Redimensionado del punto para los puntos previos
    
cv2.destroyAllWindows()
cap.realese()


AttributeError: 'cv2.VideoCapture' object has no attribute 'realese'